In [1]:
# Семинар 13: спецификация модели, тесты на выбор функциональной формы

import numpy as np
import pandas as pd # чтение файлов
import matplotlib.pyplot as plt # построение графиков
import seaborn as sns # построение графиков
import statsmodels.api as sm # тесты
import statsmodels
import statsmodels.stats.diagnostic as sm_diagnostic # тест Бройша-Пагана
import pandas as pd
from  sklearn import datasets
import statsmodels.api as sm
import statsmodels.stats.outliers_influence as oi
import scipy.stats
import statsmodels.formula.api as smf

In [2]:
get_ipython().system('pip install rdatasets') # наборы данных
# !pip install pyreadstat # чтение spss/stata данных
from rdatasets import data 
# from pyreadstat import read_sav, set_value_labels

In [3]:
# сгенерируем объясняющие переменные x1, x2 и ошибки eps

n = 100
d = pd.DataFrame({'x1': np.random.normal(100, 5, n),
                  'x2': np.random.normal(20, 3, n),
                  'eps': np.random.normal(0, 0.5, n)})
d

,x1,x2,eps
0,102.790637,17.929010,0.310458
1,96.244020,21.032717,-0.599631
2,108.650357,18.748197,1.096609
3,101.085458,20.398076,-0.352759
4,100.990297,16.549087,-0.632610
...,...,...,...
95,97.100620,17.256626,-0.035034
96,102.533258,13.018849,0.409437
97,96.807327,23.151990,-0.607418
98,93.822611,21.386406,-0.017853


In [4]:
# добавим зависимую переменную y

d['y'] = 2 + 0.7*d['x1'] - 0.3*d['x2'] + d['eps']
d

,x1,x2,eps,y
0,102.790637,17.929010,0.310458,68.885201
1,96.244020,21.032717,-0.599631,62.461368
2,108.650357,18.748197,1.096609,73.527400
3,101.085458,20.398076,-0.352759,66.287638
4,100.990297,16.549087,-0.632610,67.095872
...,...,...,...,...
95,97.100620,17.256626,-0.035034,64.758412
96,102.533258,13.018849,0.409437,70.277063
97,96.807327,23.151990,-0.607418,62.212114
98,93.822611,21.386406,-0.017853,61.242053


In [5]:
# 1. Метод Зарембки
# H0: качество подгонки линейной и полулогарифмической моделей одинаковое
# H1: модель с меньшей RSS лучше

# Для начала посчитаем среднее геометрическое y
import math
d['lny'] = np.log(d['y'])
gy = np.exp(np.mean(d['lny']))
gy # геометрическое среднее

# добавим две переменные: преобразованный y и его логарифм
d['y_new'] = d['y']/gy
d['ln_y_new'] = np.log(d['y_new'])
d

,x1,x2,eps,y,lny,y_new,ln_y_new
0,102.790637,17.929010,0.310458,68.885201,4.232441,1.047281,0.046197
1,96.244020,21.032717,-0.599631,62.461368,4.134548,0.949618,-0.051696
2,108.650357,18.748197,1.096609,73.527400,4.297658,1.117858,0.111414
3,101.085458,20.398076,-0.352759,66.287638,4.194003,1.007789,0.007759
4,100.990297,16.549087,-0.632610,67.095872,4.206123,1.020077,0.019878
...,...,...,...,...,...,...,...
95,97.100620,17.256626,-0.035034,64.758412,4.170664,0.984540,-0.015581
96,102.533258,13.018849,0.409437,70.277063,4.252445,1.068442,0.066201
97,96.807327,23.151990,-0.607418,62.212114,4.130550,0.945828,-0.055694
98,93.822611,21.386406,-0.017853,61.242053,4.114834,0.931080,-0.071410


In [6]:
# Оценим две регрессии, где зависимыми переменнными являются новый y и его логарифмиров, соответственно
model1 = smf.ols("y_new ~ x1 + x2", d).fit()
model2 = smf.ols("ln_y_new ~ x1 + x2", d).fit()

In [7]:
# Рассчитаем наблюдаемое и критическое значения:

xi_2 = n/2*np.abs(np.log(model1.ssr/model2.ssr))
print(xi_2) # наблюдаемое значение статистики
xi_crit = scipy.stats.chi2.ppf(0.95, 1)
print(xi_crit) # критическое значение (уровень значимости 5%)

# Вывод: так как наблюдаемое значение статистики больше критического, то на 5% уровне мы не можем отвергнуть гипотезу
# H1, то есть модели имеют разное качество подгонки, тогда нужно сранвить их RSS

0.38970937447710535
3.841458820694124


In [8]:
# Выведем RSS для моделей

print(model1.ssr)
print(model2.ssr)

# так как в линейной модели (model1) RSS меньше, то выбираем ее

0.005565001310465943
0.0056085454492103325


In [10]:
# Тест Харке-Бера на нормальность
# H0: остатки модели имеют нормальное распределение
# H1: распределение остатков отлично от нормального
from scipy import stats
jarque_bera_test = stats.jarque_bera(model1.resid)
jarque_bera_test

Jarque_beraResult(statistic=1.5440045387537202, pvalue=0.462086918933149)

In [11]:
# PE тест (тест для сравнения линейной и линейной в логарифмах моделей)
# можно сравнивать невложенные модели (в моделях есть набор общих регрессоров + в каждой дополнительно свои регрессоры)
# можно тестировать линейную модель против полулогарифмической, а не только против линейной в логарифмах
reg1 = smf.ols("y ~ x1 + x2", data = d).fit()
ypred = reg1.predict()

In [12]:
reg2 = smf.ols("lny ~ x1 + x2", data = d).fit()
ln_ypred = reg2.predict()

In [13]:
d['add1'] = ln_ypred - np.log(ypred)
d['add2'] = ypred - np.exp(ln_ypred)

In [14]:
reg1_add = smf.ols("y ~ x1 + x2 + add1", data = d).fit()
print(reg1_add.summary())
reg2_add = smf.ols("lny ~ x1 + x2 + add2", data = d).fit()
print(reg2_add.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.979
Model:                            OLS   Adj. R-squared:                  0.978
Method:                 Least Squares   F-statistic:                     1462.
Date:                Mon, 12 Dec 2022   Prob (F-statistic):           5.96e-80
Time:                        14:02:19   Log-Likelihood:                -70.390
No. Observations:                 100   AIC:                             148.8
Df Residuals:                      96   BIC:                             159.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.2790      1.175      2.790      0.0